# Using MongoDB Memory Tool with Strands SDK

This notebook demonstrates how to use the `mongodb_memory` tool from `strands-tools` to create an agent with persistent memory capabilities backed by MongoDB Atlas.

## Overview

The `mongodb_memory` tool provides AI agents with the ability to store and retrieve conversation context and information using MongoDB Atlas as the backend storage with vector search capabilities. This notebook shows how to:

1. Set up the Strands SDK with the mongodb_memory tool
2. Configure MongoDB Atlas connection securely
3. Create an agent that can store and retrieve memories
4. Demonstrate persistent memory across conversations
5. Show practical use cases for memory-enabled agents

## Prerequisites

Before running this notebook, ensure you have:

1. A MongoDB Atlas cluster (free tier works fine)
2. Valid MongoDB Atlas connection URI
3. AWS credentials configured for Bedrock access (required for embeddings and LLM)
4. Required Python packages installed

## Installation

First, let's install the required packages with the correct optional dependency:

In [ ]:
!pip install strands-agents "strands-agents-tools[mongodb-memory]>=0.2.16"

## Importing Libraries

In [ ]:
# Import the necessary libraries
from strands import Agent
from strands.models import BedrockModel
from strands_tools import mongodb_memory
from getpass import getpass
import os

## Setting up MongoDB Atlas Credentials

We'll securely collect MongoDB Atlas credentials using getpass to avoid hardcoding sensitive information.

**Note:** For production use, consider using environment variables or AWS Secrets Manager instead of hardcoding credentials.

In [ ]:
# Get MongoDB Atlas credentials securely
# For this demo, we'll use getpass to avoid exposing credentials
mongodb_uri = getpass("MongoDB Atlas Cluster URI: ")

# Validate that credentials are provided
if not mongodb_uri:
    raise ValueError("MongoDB Atlas Cluster URI is required")

print("✓ MongoDB Atlas credentials configured")

## Setting up the Bedrock Model

Configure the language model that will power our agent:

In [ ]:
# Create a BedrockModel
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-5-sonnet-20241022-v2:0",
    region_name='us-east-1',
    temperature=0.3,
)

print("✓ Bedrock model configured")

## Creating the Memory-Enabled Agent

Now let's create an agent that uses the mongodb_memory tool. The tool will be configured with connection parameters and used directly by the agent:

In [ ]:
# Create an agent with the mongodb_memory tool
# The tool will be configured when the agent calls it
memory_agent = Agent(
    model=bedrock_model,
    tools=[mongodb_memory],
    system_prompt="""You are a helpful assistant with persistent memory capabilities. 
    You can store important information about users and conversations using your mongodb_memory tool.
    Always try to remember key details about users, their preferences, and previous conversations.
    When appropriate, retrieve relevant memories to provide personalized responses.
    
    When using the mongodb_memory tool, use these connection parameters:
    - cluster_uri: The MongoDB Atlas connection URI provided by the user
    - database_name: "agent_memory_db"
    - collection_name: "memories"
    - namespace: "demo_session_1"
    """
)

print("✓ Memory-enabled agent created")

## Demonstrating Memory Functionality

Let's test the agent's memory capabilities with some example interactions:

### Example 1: Storing User Preferences

In [ ]:
# First interaction - introducing user preferences
response1 = memory_agent(
    f"""Hi! My name is Alice and I'm a data scientist. I love working with Python and machine learning. 
    My favorite ML framework is scikit-learn, and I prefer coffee over tea. 
    Please remember these details about me using your memory tool.
    
    Use these connection details:
    - cluster_uri: {mongodb_uri}
    - database_name: agent_memory_db
    - collection_name: memories
    - namespace: demo_session_1
    """
)

print("Agent Response:")
print(response1)

### Example 2: Retrieving Stored Information

In [ ]:
# Second interaction - testing memory recall
response2 = memory_agent(
    f"""What do you remember about me? What are my preferences? 
    Please retrieve this information from your memory.
    
    Use these connection details:
    - cluster_uri: {mongodb_uri}
    - database_name: agent_memory_db
    - collection_name: memories
    - namespace: demo_session_1
    """
)

print("Agent Response:")
print(response2)

### Example 3: Contextual Recommendations

In [ ]:
# Third interaction - using memory for personalized recommendations
response3 = memory_agent(
    f"""Based on what you know about me, can you recommend some tools and frameworks 
    I should consider for my next data science project?
    
    Use these connection details:
    - cluster_uri: {mongodb_uri}
    - database_name: agent_memory_db
    - collection_name: memories
    - namespace: demo_session_1
    """
)

print("Agent Response:")
print(response3)

### Example 4: Adding More Context

In [ ]:
# Fourth interaction - adding more information
response4 = memory_agent(
    f"""I also wanted to mention that I'm currently working on a customer churn prediction project 
    for a telecom company. The dataset has about 500,000 records. Please remember this.
    
    Use these connection details:
    - cluster_uri: {mongodb_uri}
    - database_name: agent_memory_db
    - collection_name: memories
    - namespace: demo_session_1
    """
)

print("Agent Response:")
print(response4)

### Example 5: Semantic Search Across Memories

In [ ]:
# Fifth interaction - testing semantic search
response5 = memory_agent(
    f"""What do you know about my current work projects?
    
    Use these connection details:
    - cluster_uri: {mongodb_uri}
    - database_name: agent_memory_db
    - collection_name: memories
    - namespace: demo_session_1
    """
)

print("Agent Response:")
print(response5)

## Understanding the MongoDB Memory Tool

The `mongodb_memory` tool provides several key operations:

### Available Actions:

1. **record**: Store new memories with automatic embedding generation
   - Automatically generates vector embeddings using Amazon Bedrock Titan
   - Stores content with metadata (timestamp, namespace, etc.)

2. **retrieve**: Semantic search using vector embeddings
   - Uses MongoDB Atlas Vector Search with cosine similarity
   - Filters by namespace for multi-user scenarios
   - Returns most relevant memories based on semantic similarity

3. **list**: List all memories with pagination
   - Retrieve all memories in a namespace
   - Supports pagination for large memory sets

4. **get**: Retrieve specific memories by ID
   - Direct lookup by memory ID

5. **delete**: Remove specific memories
   - Delete memories by ID

### Key Features:

- **Semantic Search**: Uses vector embeddings for intelligent memory retrieval
- **Namespace Isolation**: Separate memories for different users/sessions
- **Automatic Indexing**: Creates vector search indexes automatically
- **AWS Bedrock Integration**: Uses Titan embeddings for vector generation

## Best Practices

1. **Security**:
   - Never hardcode credentials in notebooks or code
   - Use environment variables or AWS Secrets Manager
   - Restrict MongoDB Atlas network access to your IP ranges

2. **Namespaces**:
   - Use unique namespaces for different users or sessions
   - Consider using user IDs or session IDs as namespace values

3. **Memory Management**:
   - Regularly clean up old or irrelevant memories
   - Implement memory expiration policies if needed
   - Monitor collection size and index performance

4. **Performance**:
   - MongoDB Atlas Vector Search requires proper indexing
   - The tool automatically creates indexes, but monitor their status
   - Consider using MongoDB Atlas M10+ clusters for production workloads

## Troubleshooting

### Common Issues:

1. **Connection Errors**:
   - Verify your MongoDB Atlas cluster URI is correct
   - Check network access settings in MongoDB Atlas
   - Ensure your IP is whitelisted

2. **Vector Search Not Working**:
   - Wait for vector search index to be created (can take a few minutes)
   - Check index status in MongoDB Atlas UI
   - Verify AWS credentials for Bedrock access

3. **AWS Bedrock Errors**:
   - Ensure AWS credentials are configured
   - Verify Bedrock access in your AWS region
   - Check that Titan embedding model is available in your region

## Next Steps

Now that you've learned how to use the MongoDB memory tool with Strands SDK, you can:

1. Integrate memory capabilities into your own agents
2. Experiment with different namespace strategies
3. Build multi-user applications with isolated memories
4. Combine memory with other Strands tools for more powerful agents
5. Implement custom memory management policies

For more information:
- [Strands SDK Documentation](https://github.com/strands-agents/strands)
- [MongoDB Atlas Vector Search](https://www.mongodb.com/docs/atlas/atlas-vector-search/)
- [AWS Bedrock Documentation](https://docs.aws.amazon.com/bedrock/)

## Cleanup (Optional)

If you want to clean up the memories created during this demo:

In [ ]:
# Optional: Clean up demo memories
# Uncomment the following code to delete all memories from this demo session

# from pymongo import MongoClient
# 
# client = MongoClient(mongodb_uri)
# db = client["agent_memory_db"]
# collection = db["memories"]
# 
# # Delete all memories in the demo namespace
# result = collection.delete_many({"namespace": "demo_session_1"})
# print(f"Deleted {result.deleted_count} memories")
# 
# client.close()